In [1]:
import numpy as np
import graphtools as gt
from importlib import reload
import pandas as pd
import matplotlib as plt

%matplotlib notebook

In [2]:
# Hyperparams
vfname = "veldata/secondring_t1.0v1.0l10"
info = {}
infofname = vfname+".info"
print("Creating info dict")
info = gt.get_info_dict(infofname)
for key,val in sorted(info.items()):
    print(key,"\t",val)

Creating info dict
nTG 	 144
n_drivers 	 189515
n_points 	 149165
source 	 /home/walterms/traffic/OUT0_FiveRing150buffer
tcutoff 	 1.0
tglen 	 10
velmin 	 1.0
xmax 	 9917.53639401
xmin 	 9907.53639399
ymax 	 4443.05976788
ymin 	 4431.05976788


In [3]:
reload(gt)

<module 'graphtools' from '/home/michael/msc/summer17/traffic/traffic-stgcnn/graphtools.py'>

In [4]:
# Gather nodes
xmin,xmax = info["xmin"]/gt.long2km, info["xmax"]/gt.long2km
ymin,ymax = info["ymin"]/gt.lat2km, info["ymax"]/gt.lat2km
region_gsi = [xmin,xmax,ymin,ymax]
# A combines edges and connections to their idx in senders/receivers
nodes, edges = gt.generate_nodes(region=region_gsi, mindist=0.5, maxdist=2., maxnbr=8)
print("Number of nodes", len(nodes.index))
print("Number of edges", len(edges.index))

Number of nodes 181
Number of edges 1438


In [5]:
nodes[:5]

,coords,nbrs,coords_km
0,"[116.3894407, 39.9062721]","[1, 2, 4, 13, 20, 3, 6, 91]","[9912.526188283888, 4436.678502976904]"
1,"[116.3970962, 39.9065222]","[13, 0, 6, 20, 2, 5, 14, 130]","[9913.178183377067, 4436.70630846299]"
2,"[116.3898158, 39.8987502]","[0, 20, 50, 3, 1, 57, 4, 130]","[9912.5581343827, 4435.842237140098]"
3,"[116.3779918, 39.8987474]","[4, 146, 46, 2, 49, 0, 57, 149]","[9911.55112112664, 4435.841925843172]"
4,"[116.3766593, 39.9058516]","[3, 46, 149, 0, 146, 47, 2, 89]","[9911.437636251496, 4436.63175284936]"


In [ ]:
viewer = gt.viewer(nodes,usekm=True)
viewer.connect()

In [ ]:
# Create snap files
# Each snap file should have:
# - array of node features
# - array of edge features
# - array of senders
# - array of receivers
#
# the dataframe is a great setup for this

In [79]:
reload(gt)

<module 'graphtools' from '/home/michael/msc/summer17/traffic/traffic-stgcnn/graphtools.py'>

In [15]:
# 5000 pts takes about 30 seconds
vdf = gt.get_veldf(vfname,nodedf=nodes,days=[],nTG=info["nTG"],nvel=5e3)
vdf.drop_duplicates(inplace=True)
print(len(vdf.index))

vdf[:5]

5000


,day,tg,x_km,y_km,vx,vy,v,nodeID,dist2node,angle
0,0,0,9914.82,4432.71,-12.761000,-1.71863,12.87630,154,0.290220,-3.007720
1,0,0,9914.82,4432.71,-12.761000,-1.71863,12.87630,42,0.672022,-3.007720
2,0,0,9914.82,4432.71,-12.761000,-1.71863,12.87630,125,0.776102,-3.007720
3,0,0,9915.32,4436.33,0.936032,2.85485,3.00438,127,0.443865,1.253967
4,0,0,9915.32,4436.33,0.936032,2.85485,3.00438,10,0.460241,1.253967


In [ ]:
# We have the vdf function
# Need to make an array of node features: the vavg, stddev, etc.
# Edge array of...idk
# 
# Iterate over the nodes and see which vels are involved
#
# Want to prevent rereading velocity file
# Note, vel file is not actually ordered because of the
# buffer dumping chunks
#
# Just collect all the velocity data so we can process it as df

In [12]:
# five files
node_fname = "nn_inputs/node_features"
edge_fname = "nn_inputs/edge_features"
send_fname = "nn_inputs/senders"
receive_fname = "nn_inputs/receivers"
glbl_fname = "nn_inputs/glbls"

In [38]:
# Node features:   ncars, v_avg, v_std
# Edge features:   ncars, v_avg, v_std, pol (polarity, towards/away)
# For edge features, perhaps capture cars that are within +/- pi/4 radians of the edge's line
# Don't forget about the cars going in the opposite direction too
# Picture a bowtie centered on the edge's "road line"
# For averaging, do a weighted sum based on dist2node

nodes["ncar"] = 0
nodes["v_avg"] = 0.
nodes["v_std"] = 0.

edges["ncar"] = 0
edges["v_avg_out"] = 0.
edges["v_avg_in"] = 0.
edges["v_std_out"] = 0.
edges["v_std_in"] = 0.

i_edge = 0
# for day in range(7):
#     for tg in range(info["nTG"]):
for day in [0]:
    for tg in [0]:
        # Get velstats for this day, tg
        vdf_ = vdf[(vdf['day']==day) & (vdf['tg']==tg)]
        for idx, node in nodes.iterrows():
            # Give this subset of vels, calc stats for each node
            vels = vdf_[vdf_['nodeID'] == idx]
            nodes.at[idx,'ncar'] = len(vels.index)
            if len(vels.index) == 0:
                continue
            nodes.at[idx,'v_avg'] = vels.mean(axis=0)['v']
            if len(vels.index) > 1:
                nodes.at[idx,'v_std'] = vels.std(axis=0)['v']
        
            # Iterate over this nodes edges, adding vel stats as necessary
            edges_ = edges[edges["sender"] == idx]
            for eidx, e in edges_.iterrows():
                v_out, v_in = [], []
                for iv, v in vels.iterrows():
                    dtheta = v['angle'] - e['angle']
                    if (abs(dtheta) < 0.25*np.pi) | (abs(dtheta) > 1.75*np.pi):
                        v_out.append(v['v'])
                    if (abs(dtheta) > np.pi*0.75) & (abs(dtheta) < np.pi*1.25):
                        v_in.append(v['v'])
                
                ncar = len(v_out) + len(v_in)
                edges.at[eidx, "ncar"] = ncar
                if ncar == 0:
                    continue
                if len(v_out) > 0:
                    v_avg_out, v_std_out = np.mean(v_out), np.std(v_out)
                    edges.at[eidx, "v_avg_out"] = v_avg_out
                    edges.at[eidx, "v_std_out"] = v_std_out
                if len(v_in) > 0:
                    v_avg_in, v_std_in = np.mean(v_in), np.std(v_in)
                    edges.at[eidx, "v_avg_in"] = v_avg_in
                    edges.at[eidx, "v_std_in"] = v_std_in
                
                
            

In [43]:
len(live_edges.index)

82

In [40]:
live_edges = edges[edges["ncar"] > 0]

In [42]:
edges.sum(axis=0)

sender       129294.000000
receiver     130275.000000
angle          -127.203041
ncar             98.000000
v_avg_out       441.129200
v_avg_in        396.381830
v_std_out        25.316010
v_std_in         19.690230
dtype: float64

In [29]:
len(live_edges.index)

0